# Simple Integer Linear Programming Problem

$$
\text{Max} \quad x + y +z
$$
Subject to:
$$
x + 2y + 3z \leq 4 \\
x + y \geq 1 \\
x,y, z \in \{0,1\}
$$


In [ ]:
# pip install for a limited license of the Gurobi callable library
# %pip install gurobipy

## Python Implementation

The following Python code imports the Gurobi callable library (functions and classes)

In [1]:
import gurobipy as gp
from gurobipy import GRB

# tested with Python 3.7.0 & Gurobi 9.1.0

### Creating the model
The `Model()` constructor creates a model object m . The name of this model is ‘example’. This model m initially contains no decision variables, constraints, or objective function.

In [2]:
# Create the  model m
m = gp.Model("example")

Using license file C:\Users\panit\gurobi.lic


### Adding decision variables to the model
The `m.addVar()` method adds a decision variable to the model object m, one at a time. The argument of the method gives the name of added decision variable. Each variable gets a type (binary), and a name.

In [3]:
# Create decision variables
x = m.addVar(vtype=GRB.BINARY, name="x")
y = m.addVar(vtype=GRB.BINARY, name="y")
z = m.addVar(vtype=GRB.BINARY, name="z")

### Setting the objective function
The `m.setObjective()` method adds the objective function to the model object m. The first argument is a linear expression (LinExpr) and the second argument defines the sense of the optimization. A linear expression object (LinExpr) consists of a constant term, plus a sum of coefficient-variables pairs that capture the linear terms.

In [4]:
# Define objective function
m.setObjective(x + y + 2*z, GRB.MAXIMIZE)

### Adding constraints to the model
The `m.addConstr()` method adds a constraint to the model object m and considers a linear expression (LinExpr) as the left-hand-side of the constraints, the sense of the constraint, and its RHS value. The last argument gives the name of the constraint.

In [5]:
# Constraint 1
c1 = m.addConstr(x + 2*y + 3*z <= 4, name="c1")
# Constraint 2
c2 = m.addConstr(x + y >= 1, name="c2")

In [6]:
# save model for inspection
m.write('example.lp')

![example_lp](example_lp.PNG)

The `m.optimize()` method runs the optimization engine to solve the LP problem in the model object m

In [7]:
# Run optimization engine
m.optimize()

Gurobi Optimizer version 9.1.2 build v9.1.2rc0 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 2 rows, 3 columns and 5 nonzeros
Model fingerprint: 0x98886187
Variable types: 0 continuous, 3 integer (3 binary)
Coefficient statistics:
  Matrix range     [1e+00, 3e+00]
  Objective range  [1e+00, 2e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 4e+00]
Found heuristic solution: objective 2.0000000
Presolve removed 2 rows and 3 columns
Presolve time: 0.01s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.02 seconds
Thread count was 1 (of 8 available processors)

Solution count 2: 3 2 

Optimal solution found (tolerance 1.00e-04)
Best objective 3.000000000000e+00, best bound 3.000000000000e+00, gap 0.0000%


In [8]:
# Display optimal solution
for v in m.getVars():
    print(v.varName, v.x)

print(f"\nOptimal objective function value: {m.objVal:,}")

x 1.0
y 0.0
z 1.0

Optimal objective function value: 3.0


The `m.getVars()` method retrieves a list of all variables in the model object m. The print function displays the decision variable names `m.varName` and solution value `m.x`.

## Parametrized Model

### Data

![example_data](example_data.PNG)

### Set and indices
$j \in J = \{𝑥,𝑦,𝑧 \}$: Index and set of decision variables. 

$i \in I = \{𝑐𝑜𝑛1, 𝑐𝑜𝑛2 \}$: Index and set of constraints.

### Parameters

$c_j$: Objective function coefficient associated with decision variable $j$. The objective function coefficients values are:

$$
c_x = 1, c_y = 1, c_z = 2
$$

**Note**: Observe that constraint $con2$: $x+y \geq 1$ is equivalent to   $-x-y \leq -1$.

$b_i$: RHS associated with constraint $i$. The constraints RHS values are:

$$
b_{con1} = 4, b_{con2} = -1
$$


$a_{i,j}$: coefficient of decision variable $j$ and constraint $i$. The LHS values of the constraints are:

$$
a_{con1,x} = 1, a_{con1,y} = 2, a_{con1,z} = 3 \\
a_{con2,x} = -1, a_{con2,y} = -1, a_{con2,z} = 0
$$

### Decision Variables

$v_j \in \{0,1\}$: Decision variable $j$.

### Objective Function

$$
\text{Max} \quad \sum_{j \in J} c_j * v_j
$$

### Constraints

$$
\sum_{j \in J} a_{i,j} * v_j \leq b_i \quad \forall i \in I
$$

## Create Parametrized Model

### Input Data: Sets and Parameters
We define all the input data for the model.

$c_j$: Objective function coefficient associated with decision variable $j$. The objective function coefficients values are:

$$
c_x = 1, c_y = 1, c_z = 2
$$

In [9]:
# objective function coefficients
jvar, c_j = gp.multidict({
    'x': 1,
    'y': 1,
    'z': 2
})

A Python dictionary allows you to map arbitrary key values to pieces of data. The Gurobi `multidict()` function splits a single dictionary into multiple dictionaries. The input dictionary should map each key to a list of n values. This `multidict()` function initialize the keys of the decision variable `J` and  the values of the coefficients associated to the decision variables, `c_j`.

---

$b_i$: RHS associated with constraint $i$. The constraints RHS values are:

$$
b_{con1} = 4, b_{con2} = -1
$$

In [10]:
# RHS associated to each constraint
icon, b_i = gp.multidict({
    'con1': 4,
    'con2': -1
})

This `multidict()` function initialize the keys of the constraints `I` and  the values of the RHS associated to the constraints, `b_i`.

---

$a_{i,j}$: coefficient of decision variable $j$ and constraint $i$. The LHS values of the constraints are:

$$
a_{con1,x} = 1, a_{con1,y} = 2, a_{con1,z} = 3 \\
a_{con2,x} = -1, a_{con2,y} = -1, a_{con2,z} = 0
$$

In [11]:
# Coefficients of LHS of constraints
a_ij = {
    ('con1', 'x'):  1,
    ('con1', 'y'):  2,
    ('con1', 'z'):  3,
    ('con2', 'x'):  -1,
    ('con2', 'y'):  -1,
    ('con2', 'z'):  0
}

This Python dictionary has as keys tuples with the constraint and variable associated with a LHS coefficient.

In [12]:
# multidict version (remove in final version)

ij, aa_ij = gp.multidict({
    ('con1', 'x'):  1,
    ('con1', 'y'):  2,
    ('con1', 'z'):  3,
    ('con2', 'x'):  -1,
    ('con2', 'y'):  -1,
    ('con2', 'z'):  0
})

### Creating the model
The `Model()` constructor creates a model object m2 for the parametrized version of the problem. The name of this model is ‘example2’. 

In [13]:
# Create the  model m2
m2 = gp.Model("example2")

### Decision Variables

$v_j \in \{0,1\}$: Decision variable $j$.

The `m2.addVars()` method adds all the decision variables to the model object m2.
The first argument `J` provides the indices that will be used as keys to access the variables. The second argument `vtype=GRB.BINARY` defines the decision variables as binary.
The last argument gives the name ‘v’ to the  decision variables. 

In [14]:
# Create decision variables for the set of decison variables J
v = m2.addVars(jvar, vtype=GRB.BINARY, name="v")

### Objective Function

$$
\text{Max} \quad \sum_{j \in J} c_j * v_j
$$

The `m2.setObjective()` method adds the objective function to the model object m2.
The first argument `objfcn` is a linear expression which is generated by the `gp.quicksum()` method. 
The second argument defines the sense of the optimization.

In [15]:
# Objective function
objfcn = gp.quicksum(c_j[j]*v[j] for j in jvar)

m2.setObjective(objfcn, GRB.MAXIMIZE)

### Constraints

$$
\sum_{j \in J} a_{i,j} * v_j \leq b_i \quad \forall i \in I
$$

The `addConstrs()` method adds constraints to the model object m2. We store the constraint generated in an object called `cons`. The left-hand-side of each constraint can be created by using the `gp.quicksum()` method. The second argument is the sense of the constraint (≤). 
The third argument is the RHS value associated to the constraint. 
Notice that one constraint will be generated per key in the set `icon`.
The last argument gives the name of the constraint.

In [16]:
# Create the constraints of the model

cons = m2.addConstrs(((gp.quicksum(a_ij[i,j]*v[j] for j in jvar) <= b_i[i]) for i in icon), name='cons')

In [17]:
# save model for inspection
m2.write('example2.lp')


![example2_lp](example2_lp.PNG)

In [18]:
# Run optimization engine
m2.optimize()

Gurobi Optimizer version 9.1.2 build v9.1.2rc0 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 2 rows, 3 columns and 5 nonzeros
Model fingerprint: 0x8d4960d3
Variable types: 0 continuous, 3 integer (3 binary)
Coefficient statistics:
  Matrix range     [1e+00, 3e+00]
  Objective range  [1e+00, 2e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 4e+00]
Found heuristic solution: objective 2.0000000
Presolve removed 2 rows and 3 columns
Presolve time: 0.00s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.02 seconds
Thread count was 1 (of 8 available processors)

Solution count 2: 3 2 

Optimal solution found (tolerance 1.00e-04)
Best objective 3.000000000000e+00, best bound 3.000000000000e+00, gap 0.0000%


In [19]:
# Display optimal solution
for v in m2.getVars():
    print(v.varName, v.x)

print(f"\nOptimal objective function value: {m2.objVal:,}")

v[x] 1.0
v[y] 0.0
v[z] 1.0

Optimal objective function value: 3.0


## Exercise:

Add the constraint $2x + z \leq 2$